In [59]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from arkham_scraper import get_urls_by_type, get_soup
import regex as re

In [10]:
invest, assets, events, skills = get_urls_by_type()
results = get_soup(invest[0])

Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [5]:
invest[0]

'https://arkhamdb.com/card/02005'

In [6]:
# dictionary with empty traits
    investigator_traits = {'title':[],
                           'traits':[],
                           'faction':[],  
                           'willpower':[],
                           'intellect':[],
                           'combat':[],
                           'agility':[],
                           'health':[],
                           'sanity':[],
                           'ability':[],
                           'type':[],
                           'flavor':[],
                           'artist':[],
                           'expansion':[]}

    print("Getting investigator cards...")

    # for each url get player card info from that page and add each element to skill_traits
    for url in invest:

        # make html request to arkham db and parse using BS
        results = get_soup(url)

        # get list of card elements card elements
        trait_list = get_ability_traits(results)
        
        print(f'Getting investigator card {title}...')

        # itterate through card element titles and add each to a dictionary
        for i, key in enumerate(investigator_traits):

            investigator_traits[key].append(trait_list[i])

    print("Making dataframe...")

    # convert dictionary to dataframe
    df_skill = pd.DataFrame(skill_traits)

    return df_skill

IndentationError: unexpected indent (<ipython-input-6-cecd2ffda723>, line 2)

In [8]:
def get_ability_traits(results):
    
    'title':[],
                           'traits':[],
                           'faction':[],  
                           'willpower':[],
                           'intellect':[],
                           'combat':[],
                           'agility':[],
                           'health':[],
                           'sanity':[],
                           'ability':[],
                           'deck_size':[],
                           'deckbuilding':[],
                           'intro':[],
                        
                           'type':[],
                           'flavor':[],
                           'artist':[],
                           'expansion':[]
    
    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')
    
    traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')
    
    willpower =
    
    intellect =
    
    combat = 
    
    agility =
    
    health =
    
    sanity =
    
    deck_size =
    
    deckbuilding = 
    
    intro = 

    ability = get_ability(results)

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')
    
    
    return [title, test_icons, traits, willpower, intellect, combat, agility, health, sanity, faction, ability, tipe, flavor, artist, expansion]

SyntaxError: invalid syntax (<ipython-input-8-dff859bd1c09>, line 9)

In [ ]:
    deck_size =
    
    deckbuilding = 
    
    intro = 

In [36]:
def get_stat_line(results):
    
    stat_line = results.find('div', class_='card-props').text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1], stat_list[2], stat_list[3]


def get_stamana_line(results):
    
    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]

In [11]:
title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

willpower, intellect, combat, agility = get_stat_line(results)

health, sanity = get_stamana_line(results)

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')


In [76]:
text = str(results.find_all('div', class_='card-text border-survivor')[1])

deck_size = int(re.search(r'[0-9]{2}', text)[0])

deck_size



30

In [12]:
title

'"Ashcan" Pete'

In [13]:
traits

'Drifter.'

In [14]:
faction

'Survivor'

In [15]:
tipe

'Investigator'

In [16]:
flavor

'"C\'mere, boy. We got work to do."'

In [17]:
artist

'Jake Murray'

In [19]:
expansion

'The Dunwich Legacy #5'